<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/main/dev/database/Amazon_Vine-PySpark-ETLs/PRELIM_ETL_Watches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-02 23:18:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-05-02 23:18:12 (10.9 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
import datetime
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType, DateType
from pyspark import SparkFiles

### Load Amazon Data into Spark DataFrame

Note: Enter URL for Desired Dataset

In [ ]:
#1 load product segment
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

### Create DataFrame - Perform Preliminary Cleaning

In [ ]:
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

(960872, 15)


In [ ]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|Invicta Women's 1...|         Watches|          5|            0|          0|                Y|
|   14661224| RKH8BNC3L5DLF|B00D3RGO20|Kenneth Cole New ...|         Watches|          5|            0|          0|                Y|
|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|Ritche 22mm Black...|         Watches|          2|            1|          1|                Y|
|    7211452|R31U3UH5AZ42LL|B000EQS1JW|Citizen Men's BM8...|         Watches|          5|            0|          0|                Y|
|   12733322|R2SV659OUJ945Y|B00A6GFD7S|Orient ER27009B M...|  

In [ ]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(831415, 9)


In [ ]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|Invicta Women's 1...|         Watches|          5|            0|          0|
|   14661224| RKH8BNC3L5DLF|B00D3RGO20|Kenneth Cole New ...|         Watches|          5|            0|          0|
|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|Ritche 22mm Black...|         Watches|          2|            1|          1|
|    7211452|R31U3UH5AZ42LL|B000EQS1JW|Citizen Men's BM8...|         Watches|          5|            0|          0|
|   12733322|R2SV659OUJ945Y|B00A6GFD7S|Orient ER27009B M...|         Watches|          4|            0|          0|
|    6576411| RA51CP8TR5A2L|B00EYSOSE8|Casio Men's GW-94...|         Wat

## Create Analysis-Specific DFs/Tables
### Perform Analysis-Specific Transforms

### Segmentation Analysis DF

In [ ]:
# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|    3653882|         Watches|
|   14661224|         Watches|
|   27324930|         Watches|
|    7211452|         Watches|
|   12733322|         Watches|
|    6576411|         Watches|
|   11811565|         Watches|
|   45925069|         Watches|
|   44751341|         Watches|
|    9962330|         Watches|
|   51330346|         Watches|
|    4201739|         Watches|
|   26339765|         Watches|
|    2692576|         Watches|
|   44713366|         Watches|
|   32778769|         Watches|
|   27258523|         Watches|
|   42646538|         Watches|
|   46017899|         Watches|
|   37192375|         Watches|
+-----------+----------------+
only showing top 20 rows



Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')

In [ ]:
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'watches')
#2c Check results
segment_df.show()

+-----------+-------+
|customer_id|watches|
+-----------+-------+
|    1567510|      1|
|   19502021|      1|
|   12819130|      1|
|   35329257|      2|
|     108460|      1|
|    5453476|      1|
|   29913055|      1|
|   30717305|      1|
|    1570030|      1|
|   44178035|      1|
|   26079415|      2|
|   14230926|      1|
|   43478048|      2|
|   43694941|      1|
|   12318815|      3|
|   13731855|      1|
|     740134|      1|
|   41956754|      1|
|   20324070|      3|
|   10767388|      2|
+-----------+-------+
only showing top 20 rows



In [ ]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- watches: long (nullable = false)

631959


Note: Column Name in df.sort('name' ...) must align with Column name from step 2

In [ ]:
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('watches', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-------+
|customer_id|watches|
+-----------+-------+
|   40765068|    127|
|    5072809|    105|
|   42416004|     86|
|   22962882|     66|
|   44191290|     64|
|   25160623|     59|
|   21375608|     59|
|   45112699|     59|
|    1389173|     58|
|   28244369|     55|
|    8887467|     55|
|    5956842|     53|
|   18840148|     52|
|   52716989|     52|
|   52581394|     52|
|   47769852|     51|
|   20631789|     49|
|   21148390|     48|
|   31060423|     46|
|   14726863|     45|
+-----------+-------+
only showing top 20 rows

100000


### Segmentation ETL Complete - Add Database Export Code

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cwuhtytzosg8.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", 
          "password": "FsM7eA9C4F", 
          "driver":"org.postgresql.Driver"}

Note: table name in table='name_segment' must align with table name in Postgres

In [ ]:
# Write segment table to Postgres/RDS
# 23 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='watches_segment', mode=mode, properties=config)

## Apriori Analysis DF
### Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

In [ ]:
# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|
|   14661224| RKH8BNC3L5DLF|B00D3RGO20|
|   27324930|R2HLE8WKZSU3NL|B00DKYC7TK|
|    7211452|R31U3UH5AZ42LL|B000EQS1JW|
|   12733322|R2SV659OUJ945Y|B00A6GFD7S|
|    6576411| RA51CP8TR5A2L|B00EYSOSE8|
|   11811565| RB2Q7DLDN6TH6|B00WM0QA3M|
|   45925069|R2Z6JOQ94LFHEP|B00MAMPGGE|
|   44751341| RX27XIIWY5JPB|B004LBPB7Q|
|    9962330|R15C7QEZT0LGZN|B00KGTVGKS|
|   51330346| ROTNLALUAJAUB|B00MPF0XJQ|
|    4201739|R2DYX7QU6BGOHR|B003P1OHHS|
|   26339765| RWASY7FKI7QOT|B00R70YEOE|
|    2692576|R2KKYZIN3CCL21|B000FVE3BG|
|   44713366|R22H4FGVD5O52O|B008X6JB12|
|   32778769|R11UACZERCM4ZY|B0040UOFPW|
|   27258523|R1AT8NQ38UQOL6|B00UR2R5UY|
|   42646538|R2NCZRQGIF1Q75|B00HFF57L0|
|   46017899| RJ9HWWMU4IAHF|B00F5O06E6|
|   37192375|R3CNTCKG352GL1|B00CHS398I|
+-----------+--------------+----------+
only showing top 20 rows



### Prelim Apriori ETL Complete - Add Database Export Code

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection_string>"
config = {"user":"<username>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

Note: table name in table='segment_apriori' must align with table name in Postgres

In [ ]:
# Write segment_apriori table to RDS
# 1 min 56 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='watches_apriori', mode=mode, properties=config)

## Amazon Reviews S3 -> Postgres/RDS ETL Complete



### Run Queries in Postgres to Confirm Segment ETL
Check Row Count of Segment Table - SELECT COUNT(*) FROM watches_segment;

Check 10 Rows of Segment Table - SELECT * FROM watches_segment LIMIT(10);


In [ ]:
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


### Run Queries in Postgres to Confirm Apriori ETL
Check Row Count of Apriori Table - SELECT COUNT(*) FROM watches_apriori;

Check 10 Rows of Apriori Table - SELECT * FROM watches_apriori LIMIT(10);

In [ ]:
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
